# 在word2vec上训练情感分析模型

In [5]:
import os 
import re
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from nltk.corpus import stopwords

from gensim.models.word2vec import Word2Vec

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans

In [14]:
def load_dataset(name, nrows=None):
    datasets = {
        'unlabeled_train': 'unlabeledTrainData.tsv',
        'labeled_train': 'labeledTrainData.tsv',
        'test': 'testData.tsv'
    }
    if name not in datasets:
        raise ValueError(name)
    data_file = os.path.join('.', 'data', datasets[name])
    df = pd.read_csv(data_file, sep='\t', escapechar='\\', nrows=nrows)
    print('Number of reviews: {}'.format(len(df)))
    return df

In [6]:
eng_stopwords = set(stopwords.words('english'))

def clean_text(text, remove_stopwords=False):
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    if remove_stopwords:
        words = [w for w in words if w not in eng_stopwords]
    return words

# 读入训练好的word2vec模型

In [12]:
model_name = '300features_10minwords_10context.model'
model = Word2Vec.load(os.path.join('.','model',model_name))
model

## 根据word2vec的结果多影评文本进行编码

In [15]:
df = load_dataset('labeled_train')
df.head()

Number of reviews: 25000


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [16]:
# 这里的做法是把每条评论的所有单词的词向量家和取均值，naive
def to_review_vector(review):
    words = clean_text(review)
    array = np.array([model[w] for w in words if w in model])
    return pd.Series(array.mean(axis = 0))

In [17]:
train_data_features = df.review.apply(to_review_vector)
train_data_features.head()

/Users/zhangyanan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/zhangyanan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.013136,-0.008601,0.008102,0.002320,-0.013041,0.004052,-0.008471,0.006639,0.014264,-0.019703,...,-0.003595,0.006053,-0.002416,-0.008749,-0.009902,0.011230,0.006375,-0.015609,0.009819,0.007952
1,-0.015347,0.002005,-0.000907,-0.007540,-0.012829,0.003942,-0.016349,0.011501,0.017876,-0.027880,...,-0.002300,0.010388,-0.005939,0.000633,0.002901,0.008876,0.004960,-0.010098,0.004710,0.000210
2,-0.016728,0.002448,0.000345,-0.002283,-0.006208,0.013001,-0.009454,0.013228,0.026653,-0.015238,...,0.008515,0.001848,-0.018959,0.011622,0.001687,0.007134,-0.016329,0.004105,0.000528,0.002606
3,-0.014231,-0.003985,0.004035,0.012628,-0.002745,0.003662,-0.018115,0.003086,0.014750,-0.028962,...,0.003085,0.005768,-0.017374,0.009206,0.009195,0.012355,-0.008875,-0.020016,0.004525,0.006851
4,-0.023095,-0.005043,0.006312,0.009030,-0.001652,0.009097,-0.013841,0.008905,0.018002,-0.011310,...,0.005649,0.004414,-0.011910,0.003434,-0.012384,0.004966,-0.003566,-0.000073,0.005456,0.008788


现在，每条评论都由了一个词向量代表（如果是短文本的话，应该效果还可以）

## 构建分类器

In [18]:
forest = RandomForestClassifier(n_estimators=100,random_state=20)
forest = forest.fit(train_data_features,df.sentiment)

In [19]:
# 测试一下模型是否可用
confusion_matrix(df.sentiment,forest.predict(train_data_features))

array([[12500,     0],
       [    0, 12500]])

## 清理占用内容的变量

In [20]:
del df 
del train_data_features

## 预测测试集结果并上传Kaggle

In [21]:
df = load_dataset('test')
df.head()

Number of reviews: 25000


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [22]:
test_data_features = df.review.apply(to_review_vector)
test_data_features.head()

/Users/zhangyanan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/zhangyanan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.021444,0.007004,-0.002707,0.010451,-0.004905,0.007704,-0.034731,0.021942,0.024857,-0.034567,...,0.005199,0.001436,-0.022824,0.016353,-0.002960,0.009028,-0.002879,-0.000971,-0.010807,0.004682
1,-0.014703,0.003183,0.000575,-0.006752,-0.004420,0.004441,-0.021425,0.007553,0.000617,-0.035690,...,-0.014743,0.009935,-0.008273,-0.006259,-0.013458,0.009888,0.006855,-0.016485,0.022149,0.007481
2,-0.011510,-0.022186,0.007573,-0.001629,-0.003071,-0.005469,-0.019439,0.008593,0.015717,-0.028939,...,0.000987,0.011413,-0.007867,-0.000501,-0.002767,0.011362,0.016648,-0.021968,0.010602,-0.002044
3,-0.021481,-0.007393,0.007776,-0.004275,0.007820,0.006995,-0.019880,0.006401,0.022901,-0.029515,...,-0.004158,0.004065,0.001912,-0.005680,-0.004116,0.002337,0.002194,-0.008822,0.004048,0.005908
4,-0.017962,-0.006804,0.006364,-0.007151,-0.003680,0.004489,-0.014552,0.013079,0.018630,-0.013850,...,-0.001910,0.000475,-0.006789,0.000849,-0.007021,0.016665,-0.006151,-0.010763,0.014512,-0.001965


In [23]:
result = forest.predict(test_data_features)
output = pd.DataFrame({'id':df.id,'sentiment':result})
output.to_csv(os.path.join('.','data','word2vec_model.csv'),index=False)
output.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,0
4,12128_7,1


In [24]:
del df 
del test_data_features
del forest

## 对词向量进行聚类

In [33]:
# 找到聚类的个数
word_vec = model.wv.syn0
num_clusters = word_vec.shape[0]//10

/Users/zhangyanan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [ ]:
%%time

kmeans_clustering = KMeans(n_clusters=num_clusters,n_jobs=4)
idx = kmeans_clustering.fit_predict(word_vec)